In [2]:
# imports 
%load_ext autoreload
%autoreload 2
import schillinger.rhythm as SR
import schillinger.pitch as SP
import itertools
import random
from music21 import *
%load_ext music21.ipython21

In [3]:
#notes old 

def convert2Dto1D(data):
    flat_array = []
    for d in data:
        if isinstance(d[0], list) or isinstance(d[0], tuple):
            for l in d:
                
                flat_array.append(l)
        else:
            flat_array.append(d)
    return flat_array
    

def showSplittedNotes(length, data, time_meter, note_sequece, duration_divider):
    attacks = convert2Dto1D(data)
    #TS = str(int(len(clock))) +'/'+ str(int((clock[0].duration.quarterLength*16)))
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    
    for seq, sequences in enumerate(attacks):
        p0 = stream.Part(id='part0')
        p0.timeSignature = meter.TimeSignature(TS)
        tones = []
        note_counter = 0
        for i in range(len(sequences)):
            nota = pitch.Pitch(note_sequece[note_counter%len(note_sequece)])
            note_counter += 1
            #nota.octave = 4
            nota = note.Note(nota.nameWithOctave)
            nota.duration.quarterLength = sequences[i%len(sequences)]*duration_divider
            tones.append(nota)
            
        p0.append(tones)
        s.insert(0, p0)
    return s

def showSplittedNotesWhole(length, data, time_meter, note_sequece, duration_divider):
    attacks = convert2Dto1D(data)
    #TS = str(int(len(clock))) +'/'+ str(int((clock[0].duration.quarterLength*16)))
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    
    for sequences in attacks:
        p0 = stream.Part(id='part0')
        p0.timeSignature = meter.TimeSignature(TS)
        tones = []
        note_counter = 0
        for i in range(length):
            nota = pitch.Pitch(note_sequece[note_counter%len(note_sequece)])
            note_counter += 1
            #nota.octave = 4
            nota = note.Note(nota.nameWithOctave)
            nota.duration.quarterLength = sequences[i%len(sequences)]*duration_divider
            tones.append(nota)
            
        p0.append(tones)
        s.insert(0, p0)
    return s


In [4]:
a = 4
b = 3
fraction = [a,b]

sr = SR.Type_I(fraction)
_sr_ = SR.Type_II(fraction)
sr.resultant, _sr_.resultant

([3, 1, 2, 2, 1, 3], [3, 1, 2, 1, 1, 1, 1, 2, 1, 3])

In [646]:
# notes harmony 
def showHarmony(note_sequence, myscale, rhythm, length, time_meter, duration_divider, arp=False,bass=True, drop2 = False):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
   
    p1 = stream.Part(id='part0')
    p1.timeSignature = meter.TimeSignature(TS)
    
    for es, sequences in enumerate(note_sequence):
        chords = []
        
        ### tests
        chord_seq = [sequences[0]]+sequences[1]
        
        test_chord = chord.Chord(chord_seq)
        #print(test_chord.root())
        ###
        
        c = chord.Chord(test_chord)
        c.root = sequences[0]
        c.duration.quarterLength = rhythm[es%len(rhythm)] * duration_divider
        #c.pitches[0].octave = c.pitches[0].octave-1 # drop2
        #c.pitches[2].octave = c.pitches[2].octave+1 # drop2
        #c = c.sortDiatonicAscending() # for better voice leadng but not real!
        
        if es % 2 == 0: # rotating clock counterclock positions
            #c = c.semiClosedPosition(inPlace=False,forceOctave=4)
            pass
        else:
            #c = c.closedPosition(inPlace=False,forceOctave=4)
            pass
        if drop2:
            try:
                c.pitches[0].octave = c.pitches[0].octave-1 # drop2
                c.pitches[-2].octave = c.pitches[-2].octave+1 # drop4
                pass
            except:
                pass
        
        if arp:
            for i, seq in enumerate(c):
                
                nota = seq
                nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
                chords.append(nota)
        else:
            chords.append(c)
        
        p1.append(chords)
    
   
    s.insert(0, p1)
    s = s.transpose(0)
    return s


In [627]:
# notes melody, rhythm

def clock_rotation(units):
        # Rotate Lists
        def rotate(l, x):
          return l[-x % len(l):] + l[:-x % len(l)]
        #clock
        l = []
        for i,e in enumerate(units):
            l.append(rotate(units, i))
            #print(rotate(units, i))
        return l

def showMultipleScore_MelodyHarmony(note_sequence, myscale, rhythm, length, time_meter, duration_divider, arp=False,bass=True):
    #attacks = convert2Dto1D(data)
    #TS = str(int(len(clock))) +'/'+ str(int((clock[0].duration.quarterLength*16)))
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    p0 = stream.Part(id='part0')
    p0.timeSignature = meter.TimeSignature(TS)
    p1 = stream.Part(id='part0')
    p1.timeSignature = meter.TimeSignature(TS)
    p3 = stream.Part(id='part0')
    p3.timeSignature = meter.TimeSignature(TS)
    for es, sequences in enumerate(note_sequence):
        
        
        tones = []
        chords = []
        bass_tones = []
        
        #tones
        nota = pitch.Pitch(sequences[0])
        nota.octave = nota.octave - 2
        nota = note.Note(nota.nameWithOctave)
        nota.duration.quarterLength = rhythm[es%len(rhythm)] * duration_divider
        tones.append(nota)
        
        #chords
        ### tests
        chord_seq = [sequences[0]]+sequences[1]
        
        test_chord = chord.Chord(chord_seq)
        #print(test_chord.root())
        ###
        
        c = chord.Chord(sequences[1])
        c.duration.quarterLength = rhythm[es%len(rhythm)] * duration_divider
        c = c.sortAscending(inPlace=False) # for better voice leadng but not real!
        c = c.semiClosedPosition(inPlace=False,forceOctave=4)
        try:
            c.pitches[1].octave = c.pitches[1].octave-1 # drop2
            pass
        except:
            pass
        
        if arp:
            for i, seq in enumerate(c):
                
                nota = seq
                nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
                chords.append(nota)
        else:
            chords.append(c)
            
        #bass
        whole_chord = [sequences[0]] + sequences[1]
        whole_chord = chord.Chord(whole_chord)
        nota = pitch.Pitch(whole_chord.findRoot())
        nota.octave = 2
        nota = note.Note(nota.nameWithOctave)
        nota.duration.quarterLength = rhythm[es%len(rhythm)] * duration_divider
        bass_tones.append(nota)
            
        p0.append(tones)
        p1.append(chords)
        if bass:
            p3.append(bass_tones)
        
    s.insert(0, p0)
    s.insert(0, p1)
    if bass:
        s.insert(0, p3)
    s = s.transpose(5)
    return s

# NOTES FOR RHYTM

def showSplittedNotes_RHYTHM(length, data, time_meter, note_sequece, duration_divider):
    attacks = convert2Dto1D(data)
    #TS = str(int(len(clock))) +'/'+ str(int((clock[0].duration.quarterLength*16)))
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    
    for seq, sequences in enumerate(attacks):
        p0 = stream.Part(id='part0')
        p0.timeSignature = meter.TimeSignature(TS)
        tones = []
        note_counter = 0
        for i in range(len(sequences)):
            nota = pitch.Pitch(note_sequece[note_counter%len(note_sequece)])
            note_counter += 1
            nota.octave = 4-seq
            nota = note.Note(nota.nameWithOctave)
            nota.duration.quarterLength = sequences[i%len(sequences)]*duration_divider
            tones.append(nota)
            
        p0.append(tones)
        s.insert(0, p0)
    return s


In [557]:
# SCHILLINGER INSTRUMENT DESTRIBUTION!!!
sr_1 = SR.Type_II([3,2])
sr_2 = SR.Type_II([4,3])

random_attack_combination = SR.get_random_attack_combination(sr_1.resultant)
random_attack_combination_2 = SR.get_random_attack_combination(sr_2.resultant)

fraction = 2/4
dp = SR.slice_by_fraction(sr_1.resultant, fraction) # slice the resultant for fun

durations_pattern = random_attack_combination # rhythm
attack_pattern = random_attack_combination_2 #instrument destribution

#durations_pattern = [3,1,2,1,2,1,3] # rhythm
#attack_pattern = [2,2] #instrument destribution
parts = 3 # or instuments
order_of_parts = [0,1,2]

cts = SR.CTS(parts,order_of_parts,durations_pattern,attack_pattern)
cts_pattern = cts.get_values()
        
#print(flattened)

#note_seq = [i[0] for i in cleaned_sequence]
note_seq = ["c"]
s = showSplittedNotes_RHYTHM(cts_pattern[0], cts_pattern[1], [cts_pattern[2][0],4], note_seq , 1/4)
s.show('musicxml')


In [5]:
# useful functions
# get original combinations by sum
l = _sr_.resultant
ts = 3

def get_original_combinaions_by_sum(l, ts):
    a = []
    for i in range(len(l)):
        for c in itertools.combinations(l, i):
            if sum(c) == ts and list(c) not in a:
                a.append(list(c))
    return a

ps = get_original_combinaions_by_sum(l, ts)
print(ps)
p = list(itertools.permutations(ps))
random.choice(p)
#####
result = sr.resultant
fraction = 2/4
SR.slice_by_fraction(result, fraction)
# random attack combinations!
SR.get_random_attack_combination(_sr_.resultant)
###

[[3], [1, 2], [2, 1], [1, 1, 1]]


[4, 2, 1, 8, 4]

In [555]:
# expansion tests mozart
#%reset

melody = []
scales = []

myscale = scale.HarmonicMinorScale('c4') # not needed for now

_scale_1 = ["D","E","F","G","A","B-","C#"]
# lacremosa
_notes_1 = ["D","F","C#","D", "F","A","G","A",
         "E","G","D","C#", "G","C#","C#","B-", 
         "F","D","A","D", "G","E","B-","G", 
         "A","D","E","F", "A","G","A","C#"]

_scale_2 = ["C","D","E-","F","G","A-","B-"]
# Bach Fuga Cux
_notes_2 = ["G","E-","F","G","C", "F", "E-", "D", "E-",
        "C","B-","C","D","G", "C", "B-", "A-","B-"]


### test 2 one theme multiple scales
#albinoni
_scale_1 = ["G","A","B-","C","D","E-","F#"]

_notes_1 = ["D","C","B-","A", "G", "G", "F#",
        "E-","D","C","B-", "A", "A", "G"]

_scale_2 = ["G","A","B-","C","D","E-","F"]

_notes_2 = ["G","F","G","E-", "F", "D", "E-",
        "F","E-","F","D", "E-", "C", "D"]

####

scales.append(_scale_1)
melody.append(_notes_1)

scales.append(_scale_2)
melody.append(_notes_2)

#insert modulation
common_notes = list(set(melody[0]).intersection(melody[1]))
melody.insert(1, common_notes)
scales.insert(1, common_notes)

##################### OWN SCALE AS MELODY

#as numbers
scales = [[9,10,0,1,3,4,6,7]]

# importing own melodies test
sBach = converter.parse('midis/test.xml') 
partOnePitches = sBach.parts[0].pitches
melody = [[p.name for p in partOnePitches]]
scales = melody

#########

sr = SR.Type_II([4,3])
seq_rhythm = _sr_.resultant

#seq_rhythm = [3,2,1,2,1,6,3]

#notes_permutations = SPG.melodic_forms_definition(notes)
voice_amount = 4
theme_expansion = 1
scale_expansion_amount = 1 # # 1st order expansion for harmonizing is according to schillingers default

whole_seq = SP.get_whole_sequence(melody, scales, voice_amount, theme_expansion, scale_expansion_amount)
merged_seq = list(itertools.chain(*whole_seq))

s = showMultipleScore_MelodyHarmony(merged_seq, myscale, seq_rhythm, len(merged_seq)*2, [3,4], 1/4, arp=False, bass=True)
#sp = midi.realtime.StreamPlayer(s)
#sp.play()
#s.show('midi')
s.show('musicxml')
#s.show()

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [136]:
# importing own melodies
sBach = converter.parse('midis/test.xml') 
partOnePitches = sBach.parts[0].pitches
center = 3
[p.pitchClass for p in partOnePitches];



#sBach.show()  

In [683]:
## PrimaryAxis ## the primary axises can change but this class describes 1 PA for now
#### so 0 can vary!
# what kind of data structure should i use?? scale?

# primary axis PA
# motions in PA = 0,A,B,C and combinations of them
# AXES:
# 0 = no motion = absolute balance
# A = Accending from PA = unbalancing (leading away from 0)
# B = Descending toward PA = balancing (leading to 0)
# C = Accending towards PA = balancing (leading to 0)
# D = Descending from PA = unbalancing (leading away from 0)
# VARIABLES:
# two variables = P for pitch and T for time
# f.e.: ATP
# f.e.2: A4T3P

# generate PA class
# 4 args scale, motion sequence, P, T
# take PAaxes combination f.e. [2xA 1xB 3xD 1xC] or [AABDDDC]
# take variables f.e. P = [2,1,1,2]
# P: f.e. P = [2,1,1,2]
# T: f.e. T = [3,1,2,2,2,3] or so
# combine PAaxes with P and T
# return PA melody

class PrimaryAxis:
    
    def __init__(self, harmony_length, motion_sequence, T, P):
        
        self.sequence_list = [] # not needed in return
        self.durations = [] # return
        self.pitches = [] # return
        self.seq_length = 0 # return
        
        self.harmony_length = len(motion_sequence)## !
        self.motion_sequence = motion_sequence
        self.P = P
        self.T = T
        
        self.combinePA()
        self.calculate_sequence()
        self.seq_length = len(self.durations)
    
    def combinePA(self):
        
        for i in range(self.harmony_length):
            sequence = self.motion_sequence[i],self.T[i%len(self.T)],self.P[i%len(self.P)]
            self.sequence_list.append(sequence)
        return
    
    def calculate_sequence(self):
        for i, e in enumerate(self.sequence_list):
            if e[0] == "A":
                self.durations.append(e[1])
                self.pitches.append([0, 0 + e[2]])
            elif e[0] == "B":
                self.durations.append(e[1])
                self.pitches.append([0 + e[2], 0])
            elif e[0] == "C":
                self.durations.append(e[1])
                self.pitches.append([0 - e[2],0])
            elif e[0] == "D":
                self.durations.append(e[1])
                self.pitches.append([0, 0 - e[2]])
            elif e[0] == "0":
                self.durations.append(e[1])
                self.pitches.append([0, 0])

def convert_to_music21_sequence(sc, pitches, durations):
    note_seq = []
    tonic = sc.getTonic()
    for i, o in enumerate(pitches):
        for e, p in enumerate(o):
            #c = sc.pitchFromDegree(p+1)
            if p != 0:
                c = sc.next(tonic, direction=p)
            else:
                c = tonic
                
            nota = note.Note(c.nameWithOctave)
            if p >= 0:
                nota.octave = 4
            else:
                nota.octave = 3
            if e is 0:
                nota.duration.quarterLength = durations[i]
            else:
                nota.duration.quarterLength = 1
            note_seq.append(nota)
    return note_seq



In [684]:
# motion_sequence = "AABDDDC"

motion_sequence = "DDBCCADDABCDAC"
time_seqence = [4,2,3,1]
pitch_sequence = [3,2,1,1,2,3]
harmony_length = 8 #not needed for now
PA = PrimaryAxis(harmony_length, motion_sequence, time_seqence, pitch_sequence)

myscale = scale.ConcreteScale(pitches = ["C4", "D4", "E-4", "F4", "G4", "A-4", "B-4"]) 

pitches = convert_to_music21_sequence(myscale,PA.pitches, PA.durations)
scales = [[myscale.pitchFromDegree(x+1).name for x in range(7)]]

melody = [[p.name for p in pitches]]
scales = [[myscale.pitchFromDegree(x+1).name for x in range(7)]]
seq_rhythm = [p.duration.quarterLength for p in pitches] #for own melody test

In [634]:
# diatonic harmony test
#test_scale = ["C4", "D4", "E-4", "F4", "G4", "A-4", "B-4"]
myscale_2 = scale.LydianScale("C4")
test_scale = [str(p) for p in myscale_2.pitches[:-1]]
SPG = SP.PitchGroup()
scale_expansion = SPG.expansions(test_scale)
C7 = scale_expansion[0]
C3 = scale_expansion[4]
C5 = scale_expansion[2]
diatonic_cycle = SP.diatonic_cicles_with_coeficients(test_scale,[C7,C5,C3,C7,C5,C3],[3,1,3,1,3,1])
#print(diatonic_cycle)
melody = [diatonic_cycle]
scales = [test_scale]


sr = SR.Type_II([4,3])
#seq_rhythm = _sr_.resultant
seq_rhythm = [1]

In [688]:
#'with modulation'
SPG = SP.PitchGroup()
scales = []
melody = []
#### Lydian
myscale_2 = scale.HarmonicMinorScale("C4")
test_scale = [str(p) for p in myscale_2.pitches[:-1]]

scale_expansion = SPG.expansions(test_scale)
C7 = scale_expansion[0]
C3 = scale_expansion[4]
C5 = scale_expansion[2]
diatonic_cycle = SP.diatonic_cicles_with_coeficients(test_scale,[C5,C3,C7],[2,1,1,2])
#diatonic_cycle = SR.slice_by_fraction(diatonic_cycle, 2/3)
scales.append(test_scale)
melody.append(diatonic_cycle)
print(diatonic_cycle)
####

#### dorian
myscale_2 = scale.DorianScale("C4")
test_scale = [str(p) for p in myscale_2.pitches[:-1]]
scale_expansion = SPG.expansions(test_scale)
C7 = scale_expansion[0]
C3 = scale_expansion[4]
C5 = scale_expansion[2]
diatonic_cycle = SP.diatonic_cicles_with_coeficients(test_scale,[C5,C3,C7],[2,1,1,2])
#diatonic_cycle = SR.slice_by_fraction(diatonic_cycle, 2/3)
scales.append(test_scale)
melody.append(diatonic_cycle)
print(diatonic_cycle)
####

#### common_notes
c = chord.Chord(list(set(scales[0]).intersection(scales[1])))
c = c.sortAscending()
#print(c.sortAscending(), c.pitches)
myscale_2 = scale.ConcreteScale(pitches = c.pitches)
#print([str(p) for p in myscale_2.pitches[:-1]])
test_scale = [str(p) for p in myscale_2.pitches[:-1]]
scale_expansion = SPG.expansions(test_scale)
C7 = scale_expansion[0]
C3 = scale_expansion[4%len(scale_expansion)]
C5 = scale_expansion[2%len(scale_expansion)]
diatonic_cycle = SP.diatonic_cicles_with_coeficients(test_scale,[C7],[1])
#diatonic_cycle = SR.slice_by_fraction(diatonic_cycle, 2/3)
print(diatonic_cycle)
scales.insert(1,test_scale)
melody.insert(1,diatonic_cycle)
print(diatonic_cycle)
#insert modulation
#common_notes = list(set(melody[0]).intersection(melody[1]))
#melody.insert(1, common_notes)
#scales.insert(1, common_notes)


#rhathm

seq_rhythm = [1]

['C4', 'F4', 'B4', 'G4', 'A-4', 'D4', 'G4', 'C4', 'F4', 'D4', 'E-4', 'A-4', 'D4', 'G4', 'C4', 'A-4', 'B4', 'E-4', 'A-4', 'D4', 'G4', 'E-4', 'F4', 'B4', 'E-4', 'A-4', 'D4', 'B4', 'C4', 'F4', 'B4', 'E-4', 'A-4', 'F4', 'G4', 'C4', 'F4', 'B4', 'E-4', 'C4', 'D4', 'G4', 'C4']
['C4', 'F4', 'B-4', 'G4', 'A4', 'D4', 'G4', 'C4', 'F4', 'D4', 'E-4', 'A4', 'D4', 'G4', 'C4', 'A4', 'B-4', 'E-4', 'A4', 'D4', 'G4', 'E-4', 'F4', 'B-4', 'E-4', 'A4', 'D4', 'B-4', 'C4', 'F4', 'B-4', 'E-4', 'A4', 'F4', 'G4', 'C4', 'F4', 'B-4', 'E-4', 'C4', 'D4', 'G4', 'C4']
['C4', 'D4', 'E-4', 'F4', 'G4', 'C4']
['C4', 'D4', 'E-4', 'F4', 'G4', 'C4']


In [692]:
# music21 converter 

########################################################
#notes_permutations = SPG.melodic_forms_definition(notes)
voice_amount = 3
theme_expansion = 0
scale_expansion_amount = 1 # # 1st order expansion for harmonizing is according to schillingers default

#whole_seq = SP.get_whole_sequence(melody, scales, voice_amount, theme_expansion, scale_expansion_amount)
whole_seq = SP.get_whole_sequence_bass(melody, scales, voice_amount, theme_expansion, scale_expansion_amount)
merged_seq = list(itertools.chain(*whole_seq))
#print(whole_seq)
## apply original melody (correct octaves)
for i, m in enumerate(merged_seq):
    #merged_seq[i][0] = merged_seq[i][0] + str(pitches[i].octave)
    pass
    
#s = showMultipleScore_MelodyHarmony(merged_seq, myscale, seq_rhythm, len(merged_seq)*2, [4,4], 2, arp=False, bass=False)
s = showHarmony(merged_seq, myscale, seq_rhythm, len(merged_seq)*2, (4,4), 1, arp=True, bass=False, drop2=True)


s.show('musicxml')

In [691]:
# tests
test_scale = ["C4", "D4", "E4", "F4", "G4", "A4", "B4"]
SPG = SP.PitchGroup()
scale_expansion = SPG.expansions(test_scale)
C7 = scale_expansion[0]
C3 = scale_expansion[4]
C5 = scale_expansion[2]
diatonic_cycle = SP.diatonic_cicles_with_coeficients(test_scale,[C7,C5,C3,C7,C5,C3],[3,1,3,1,3,1])
#print(diatonic_cycle, len(diatonic_cycle))


In [597]:
dir(scale)

['AbstractCyclicalScale',
 'AbstractDiatonicScale',
 'AbstractHarmonicMinorScale',
 'AbstractMelodicMinorScale',
 'AbstractOctatonicScale',
 'AbstractOctaveRepeatingScale',
 'AbstractRagAsawari',
 'AbstractRagMarwa',
 'AbstractScale',
 'AbstractWeightedHexatonicBlues',
 'ChromaticScale',
 'ConcreteScale',
 'CyclicalScale',
 'DIRECTION_ASCENDING',
 'DIRECTION_BI',
 'DIRECTION_DESCENDING',
 'DiatonicScale',
 'DorianScale',
 'HarmonicMinorScale',
 'HypoaeolianScale',
 'HypodorianScale',
 'HypolocrianScale',
 'HypolydianScale',
 'HypomixolydianScale',
 'HypophrygianScale',
 'LocrianScale',
 'LydianScale',
 'MajorScale',
 'MelodicMinorScale',
 'MinorScale',
 'MixolydianScale',
 'OctatonicScale',
 'OctaveRepeatingScale',
 'PhrygianScale',
 'RagAsawari',
 'RagMarwa',
 'ScalaScale',
 'Scale',
 'ScaleException',
 'SieveScale',
 'TERMINUS_HIGH',
 'TERMINUS_LOW',
 'Test',
 'WeightedHexatonicBlues',
 'WholeToneScale',
 '_DOC_ORDER',
 '_MOD',
 '__ALL__',
 '__builtins__',
 '__cached__',
 '__doc__',
